In [0]:
import pyspark.sql.functions as F

In [0]:
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [0]:
rdd = sc.parallelize(numbers)

In [0]:
rdd.getNumPartitions()

Out[30]: 8

In [0]:
rdd.glom().collect()

Out[31]: [[1], [2, 3], [4], [5, 6], [7], [8, 9], [10], [11, 12]]

In [0]:
rdd2 = rdd.coalesce(1)

In [0]:
rdd2.getNumPartitions()

Out[33]: 1

In [0]:
rdd2.glom().collect()

Out[34]: [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]

In [0]:
rdd3 = rdd.repartition(4)

In [0]:
rdd3.getNumPartitions()

Out[36]: 4

In [0]:
rdd3.glom().collect()

Out[37]: [[4, 5, 6, 10], [2, 3, 8, 9, 11, 12], [7], [1]]

In [0]:
df = spark.read.option("delimiter",";").option("header","true").csv("/FileStore/tables/carga_jan/covid_basico.csv")

In [0]:
df.rdd.getNumPartitions()

Out[39]: 1

In [0]:
df_partition = df.repartition(F.col("regiao"))

In [0]:
df_partition.rdd.glom().collect()

Out[41]: [[Row(regiao='Nordeste', estado='MA', data='2020-01-30', casosNovos='0', casosAcumulados='0', obitosNovos='0', obitosAcumulados='0'),
  Row(regiao='Nordeste', estado='MA', data='2020-01-31', casosNovos='0', casosAcumulados='0', obitosNovos='0', obitosAcumulados='0'),
  Row(regiao='Nordeste', estado='MA', data='2020-02-01', casosNovos='0', casosAcumulados='0', obitosNovos='0', obitosAcumulados='0'),
  Row(regiao='Nordeste', estado='MA', data='2020-02-02', casosNovos='0', casosAcumulados='0', obitosNovos='0', obitosAcumulados='0'),
  Row(regiao='Nordeste', estado='MA', data='2020-02-03', casosNovos='0', casosAcumulados='0', obitosNovos='0', obitosAcumulados='0'),
  Row(regiao='Nordeste', estado='MA', data='2020-02-04', casosNovos='0', casosAcumulados='0', obitosNovos='0', obitosAcumulados='0'),
  Row(regiao='Nordeste', estado='MA', data='2020-02-05', casosNovos='0', casosAcumulados='0', obitosNovos='0', obitosAcumulados='0'),
  Row(regiao='Nordeste', estado='MA', data='2020-02-0

In [0]:
df_repartition2 = df_partition.repartition(200)

In [0]:
df_repartition2.rdd.glom().collect()

Out[43]: [[Row(regiao='Norte', estado='AM', data='2020-02-16', casosNovos='0', casosAcumulados='0', obitosNovos='0', obitosAcumulados='0'),
  Row(regiao='Nordeste', estado='SE', data='2020-01-30', casosNovos='0', casosAcumulados='0', obitosNovos='0', obitosAcumulados='0'),
  Row(regiao='Norte', estado='AP', data='2020-01-31', casosNovos='0', casosAcumulados='0', obitosNovos='0', obitosAcumulados='0'),
  Row(regiao='Nordeste', estado='MA', data='2020-04-15', casosNovos='152', casosAcumulados='630', obitosNovos='2', obitosAcumulados='34'),
  Row(regiao='Norte', estado='AM', data='2020-02-09', casosNovos='0', casosAcumulados='0', obitosNovos='0', obitosAcumulados='0'),
  Row(regiao='Nordeste', estado='PI', data='2020-03-12', casosNovos='0', casosAcumulados='0', obitosNovos='0', obitosAcumulados='0'),
  Row(regiao='Sudeste', estado='SP', data='2020-04-13', casosNovos='140', casosAcumulados='8895', obitosNovos='20', obitosAcumulados='608'),
  Row(regiao='Sudeste', estado='SP', data='2020-02

In [0]:
df_repartition2.rdd.getNumPartitions()

Out[44]: 200

In [0]:
def print_partitions(df):
    numPartitions = df.rdd.getNumPartitions()
    print("Total partitions: {}".format(numPartitions))   
    df.explain()
    parts = df.rdd.glom().collect()
    i = 0
    j = 0
    for p in parts:
        print("Partition {}:".format(i))
        for r in p:
            print("Row {}:{}".format(j, r))
            j = j+1
        i = i+1


In [0]:
print_partitions(df_repartition2)

Total partitions: 200
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   ShuffleQueryStage 0, Statistics(sizeInBytes=296.3 KiB, rowCount=2.35E+3, isRuntime=true)
   +- Exchange RoundRobinPartitioning(200), REPARTITION_BY_NUM, [id=#546]
      +- FileScan csv [regiao#522,estado#523,data#524,casosNovos#525,casosAcumulados#526,obitosNovos#527,obitosAcumulados#528] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[dbfs:/FileStore/tables/carga_jan/covid_basico.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<regiao:string,estado:string,data:string,casosNovos:string,casosAcumulados:string,obitosNov...
+- == Initial Plan ==
   Exchange RoundRobinPartitioning(200), REPARTITION_BY_NUM, [id=#546]
   +- FileScan csv [regiao#522,estado#523,data#524,casosNovos#525,casosAcumulados#526,obitosNovos#527,obitosAcumulados#528] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[dbfs:/FileStore/table

In [0]:
df_repartition2.write.partitionBy("regiao","estado").mode("overwrite").csv("/FileStore/tables/carga_jan/exemplo/")

In [0]:
 df = (df.withColumn("Dia", F.substring(F.col("data"),9,2).cast("integer")).
          withColumn("Mes", F.substring(F.col("data"), 6, 2).cast("integer")).
          withColumn("Ano", F.substring(F.col("data"), 1, 4).cast("integer")))

In [0]:
df.write.partitionBy("mes","dia").mode("overwrite").csv("/FileStore/tables/carga_jan/exemplo02/")